## 1. Import Necessary Libraries

In [1]:
from transformers import AutoProcessor, Gemma3nForConditionalGeneration
from PIL import Image
import torch
import os
from tqdm.auto import tqdm
import pandas as pd

## 2. Load the Model and Processor

This block handles the setup of the core components: the pre-trained model and its associated processor.

* **`model_id`**: We specify the identifier for the model we want to use from the Hugging Face Hub, which in this case is `"google/gemma-3n-e2b-it"`.
* **Model Loading**: The `Gemma3nForConditionalGeneration.from_pretrained()` method downloads and initializes the model.
    * `torch_dtype=torch.bfloat16`: This loads the model's weights using `bfloat16` precision. It's a memory-efficient format that can speed up computation on compatible GPUs.
* **Processor Loading**: `AutoProcessor.from_pretrained()` automatically fetches the correct processor for the specified model. The processor is responsible for converting our raw inputs (like text and images) into the numerical tensors the model requires.

In [2]:
# Define the identifier for the pre-trained model on the Hugging Face Hub.
model_id = "google/gemma-3n-e2b-it"
# Load the pre-trained model from the specified ID.
model = Gemma3nForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16,).cuda().eval()
# Load the appropriate processor associated with the model.
processor = AutoProcessor.from_pretrained(model_id)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## 3. Automated Image Processing and Inference

This part automates the process of running inference on a directory of images. It iterates through each image and asks a series of predefined questions, then stores the model's responses.

### 3.1. Initialization and Setup
First, we initialize the necessary variables:
* **`results_df` & `data`**: A pandas DataFrame and a list are created to store the final results. We'll append results to the `data` list first for efficiency, then convert it to a DataFrame later.
* **File Paths**: The script defines the path to the folder containing the images and lists all files within it.
* **Prompt Generation**: A list of `prompts` is dynamically created. It starts with one general question ("What do you see on the table?") and then programmatically adds specific questions for each item in the `objects` list to test the model's object recognition capabilities.

### 3.2. Main Inference Loop
The code uses a nested loop structure:
1.  **Outer Loop**: Iterates through every `file` in the image folder, wrapped in `tqdm` to show a progress bar.
2.  **Inner Loop**: For each image, it iterates through every `prompt` in our generated list.

### 3.3. Preparing Model Inputs
Inside the loop, for each image-prompt pair, we construct the input for the model in a specific conversational format:
* **`messages`**: A list of dictionaries that defines the conversation history.
    * **`role: "system"`**: This sets the model's persona and gives it high-level instructions, like ignoring the mechanical hand in the images.
    * **`role: "user"`**: This contains the user's input, which is multi-modal (containing both an image and text). The image is opened with PIL, and the corresponding text prompt is included.

### 3.4. Generating and Decoding the Response
* **`processor.apply_chat_template`**: This function takes the `messages` list and converts it into the numerical tensor format that the model can understand.
* **`model.generate`**: This is the core inference step where the model generates a response based on the input. We use `do_sample=False` for deterministic (non-random) output.
* **Decoding**: The generated output (which is a sequence of token IDs) is sliced to isolate only the new tokens and then decoded back into a human-readable string using `processor.decode`.

### 3.5. Storing Results
The `filename`, `prompt`, the model's `response`, and an empty `Ground Truth` column are stored in a dictionary and appended to the `data` list for later analysis.

In [3]:
# Create a DataFrame to store the results
results_df = pd.DataFrame(columns=["filename", "prompt", "response","Ground Truth"])
# Initialize an empty list to temporarily hold the results before adding them to the DataFrame.
data=[]
# Define the path to the folder containing the images to be processed.
folder_path = "Images"
files = os.listdir(folder_path)

# Define a list of objects to specifically ask the model about.
objects=["cube","controller","tape","camera","keyboard","scissors","glue","screwdriver","pencil","tablet bottle","Caliper","glasses"]

# Initialize a list of prompts with a general question.
prompts = ["What do you see on the table?"]
# Dynamically create and add specific "Yes/No" questions for each object.
for obj in objects:
    prompts.append(f"I can’t find {obj} do you see it? Answer with Yes or No.")

for file in tqdm(files):
    prompt = f"What do you see on the table?"
    for prompt in prompts:
        print(f"Processing {file} with prompt: {prompt}")

        # Construct the message payload for the model in a conversational format.
        messages = [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a helpful assistant for a user operating a mechanical hand. Answer questions clearly and provide relevant information based on user needs. If a mechanical hand is visible in the image, ignore it and focus on describing or analyzing other parts of the image as requested."}]
            },
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": Image.open(folder_path+"/"+file).convert("RGB")},
                    {"type": "text", "text": prompt}
                ]
            }
        ]
        # Use the processor to apply the chat template, tokenize, and convert to PyTorch tensors.
        inputs = processor.apply_chat_template(
            messages,
            add_generation_prompt=True, 
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to(model.device, dtype=torch.bfloat16)

        # Get the length of the input tokens to later separate it from the generated output.
        input_len = inputs["input_ids"].shape[-1]

        with torch.inference_mode():
            # Generate a response from the model.
            generation = model.generate(**inputs, max_new_tokens=256, do_sample=False)
            # Slice the output tensor to get only the newly generated tokens.
            generation = generation[0][input_len:]
        # Decode the generated token IDs back into a human-readable string.
        decoded = processor.decode(generation, skip_special_tokens=True)

        # Append the results of this run to our data list.
        data.append({"filename": file.split('.')[0], "prompt": prompt, "response": decoded,"Ground Truth": ""})
        print(decoded)

  0%|          | 0/22 [00:00<?, ?it/s]

Processing 00.png with prompt: What do you see on the table?


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


On the table, I see a light-colored wooden surface with a blue and white checkered pattern underneath. There is a small, green cube on the table, and a black rectangular container with some contents inside it.
Processing 00.png with prompt: I can’t find cube do you see it? Answer with Yes or No.


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

No.
Processing output_image.png with prompt: I can’t find tablet bottle do you see it? Answer with Yes or No.


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


No.
Processing output_image.png with prompt: I can’t find Caliper do you see it? Answer with Yes or No.


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


No.
Processing output_image.png with prompt: I can’t find glasses do you see it? Answer with Yes or No.
No.


In [ ]:
# Convert the list of results into a DataFrame and save it to an Excel file.
data_df = pd.DataFrame(data)
data_df.to_excel("output.xlsx", index=False)

## 4. Calculating and Summarizing Model Accuracy

This final part is used to analyze the model's performance after the results have been manually reviewed and annotated.

### 4.1. Manual Annotation Prerequisite
First, the data generated by the previous code is saved to an Excel file. A human then manually reviews each response from the model. Based on the image and the prompt, the **`Ground Truth`** column is filled in:
* **1** is entered if the model's response is correct.
* **0** is entered if the model's response is incorrect.

This manual step is crucial because evaluating the correctness of a generative model often requires human judgment.

### 4.2. Performance Analysis
then reads this annotated Excel file to calculate the accuracy for each image:
1.  **Group Data**: It groups the data by **`filename`**.
2.  **Count Occurrences**: For each image, it counts the number of correct (`1`) and incorrect (`0`) answers using `value_counts()`.
3.  **Restructure Data**: The `.unstack()` method pivots the table, creating separate columns for the counts of `0`s and `1`s, making it easier to work with.
4.  **Calculate Accuracy**: A new **`Accuracy`** column is calculated using the formula: `(Correct Answers / Total Answers) * 100`.

In [ ]:
import pandas as pd

df = pd.read_excel('output.xlsx')

In [ ]:
result = df.groupby('filename')['Ground Truth'].value_counts().unstack(fill_value=0)

result.columns = ['Count_0', 'Count_1']
result['Accuracy'] = (result['Count_1'] / (result['Count_0'] + result['Count_1']) * 100).round(2)
result = result.reset_index()

print(result)